In [1]:
import os
import requests
import json
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np
from datetime import datetime

In [2]:
coin = 'bitcoin'

load_dotenv()

url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
days = 1
params = {
    "vs_currency": "usd",
    "days": days
}

print(params)

{'vs_currency': 'usd', 'days': 1}


# EXTRACT

In [5]:

response = requests.get(url, params=params)

payload = response.json()
payload.keys()


dict_keys(['prices', 'market_caps', 'total_volumes'])

# TRANSFORM

In [6]:
prices = payload.get("prices", [])
market_caps = payload.get("market_caps", [])
volumes = payload.get("total_volumes", [])

prices[0]


[1758896264030, 109354.21980111726]

In [12]:
df_prices = pd.DataFrame(prices, columns=["timestamp", "price"])
df_mc = pd.DataFrame(market_caps, columns=["timestamp", "market_cap"])
df_vol = pd.DataFrame(volumes, columns=["timestamp", "volume"])

df_prices

,timestamp,price
0,1758673102340,112090.131880
1,1758673416365,112352.522951
2,1758673542886,112328.789758
3,1758673904718,112293.485270
4,1758674112489,112258.417118
...,...,...
283,1758758522717,113317.162689
284,1758758873107,113435.601406
285,1758759073077,113492.312127
286,1758759420574,113476.713120


In [13]:
df = df_prices.merge(df_mc, on="timestamp").merge(df_vol, on="timestamp")

df

,timestamp,price,market_cap,volume
0,1758673102340,112090.131880,2.231548e+12,4.632238e+10
1,1758673416365,112352.522951,2.235440e+12,4.626262e+10
2,1758673542886,112328.789758,2.235440e+12,4.616737e+10
3,1758673904718,112293.485270,2.237613e+12,4.608641e+10
4,1758674112489,112258.417118,2.237613e+12,4.630285e+10
...,...,...,...,...
283,1758758522717,113317.162689,2.257709e+12,4.652471e+10
284,1758758873107,113435.601406,2.257709e+12,4.658710e+10
285,1758759073077,113492.312127,2.257709e+12,4.511225e+10
286,1758759420574,113476.713120,2.261289e+12,4.645809e+10


In [14]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
df["coin_id"] = coin

df = df[["coin_id", "timestamp", "price", "market_cap", "volume"]]

df


,coin_id,timestamp,price,market_cap,volume
0,bitcoin,2025-09-24 00:18:22.340,112090.131880,2.231548e+12,4.632238e+10
1,bitcoin,2025-09-24 00:23:36.365,112352.522951,2.235440e+12,4.626262e+10
2,bitcoin,2025-09-24 00:25:42.886,112328.789758,2.235440e+12,4.616737e+10
3,bitcoin,2025-09-24 00:31:44.718,112293.485270,2.237613e+12,4.608641e+10
4,bitcoin,2025-09-24 00:35:12.489,112258.417118,2.237613e+12,4.630285e+10
...,...,...,...,...,...
283,bitcoin,2025-09-25 00:02:02.717,113317.162689,2.257709e+12,4.652471e+10
284,bitcoin,2025-09-25 00:07:53.107,113435.601406,2.257709e+12,4.658710e+10
285,bitcoin,2025-09-25 00:11:13.077,113492.312127,2.257709e+12,4.511225e+10
286,bitcoin,2025-09-25 00:17:00.574,113476.713120,2.261289e+12,4.645809e+10
